In [0]:
%run "./startup"

<p style="display: flex; gap: 10px;">
  <a href="https://adb-3494755435873649.9.azuredatabricks.net/editor/files/1728975736729857?o=3494755435873649" target="_blank">
    <img src="/Workspace/Repos/APPS/SRC/source/dbx-mitdataprocesses/MITAFO/ATRAN_0050/Jobs/PATRIF_01/08_GEN_MOVIMIENTOS/02_TRANSFERENCIAS/INFONAVIT/Notebooks/assets/NB_PATRIF_MOV_TRANS_TA_01.png" style="max-width: 100%; height: auto;"/>
  </a>
</p>

In [0]:
params = WidgetParams(
    {
        "sr_actualiza": str,
        "sr_etapa": str,
        "sr_fec_acc": str,
        "sr_fec_liq": str,
        "sr_folio": str,
        "sr_id_archivo": str,
        "sr_id_snapshot": str,
        "sr_instancia_proceso": str,
        "sr_paso": str,
        "sr_proceso": str,
        "sr_reproceso": str,
        "sr_subetapa": str,
        "sr_subproceso": str,
        "sr_usuario": str,
    }
)
params.validate()
conf = ConfManager()
query = QueryManager()
db = DBXConnectionManager()
queries_df = query.get_sql_list()
# filtar solo los registros donde la columna 'Archivo SQL' contenga el nombre del notebook
display(queries_df.filter(col("Archivo SQL").contains("PATRIF_MOV_TRANS_TA_01")))

<p style="display: flex; gap: 10px;">
  <a href="https://adb-3494755435873649.9.azuredatabricks.net/editor/files/1728975736729869?o=3494755435873649" target="_blank">
    <img src="/Workspace/Repos/APPS/SRC/source/dbx-mitdataprocesses/MITAFO/ATRAN_0050/Jobs/PATRIF_01/08_GEN_MOVIMIENTOS/02_TRANSFERENCIAS/INFONAVIT/Notebooks/assets/NB_PATRIF_MOV_TRANS_TA_01_A.png" style="max-width: 100%; height: auto;"/>
  </a>
</p>

In [0]:
statement_001 = query.get_statement(
    "PATRIF_MOV_TRANS_TA_01_01.sql",
    CATALOG_SCHEMA = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}",
    DELTA_700_SALDOS_PROC = f"{conf.delta_700_saldos_proc}_{params.sr_folio}",
    p_folio = params.sr_folio,
    p_proceso = params.sr_proceso,
    p_tipo_subcta15 = conf.p_tipo_subcta15,
    p_tipo_subcta16 = conf.p_tipo_subcta16,
)

In [0]:
db.write_delta(f"DELTA_TMP_MOV_TRANS_SALDOS_{params.sr_folio}", db.sql_delta(statement_001), "overwrite")
if conf.debug:
    display(db.read_delta(f"DELTA_TMP_MOV_TRANS_SALDOS_{params.sr_folio}"))

<p style="display: flex; gap: 10px;">
  <a href="https://adb-3494755435873649.9.azuredatabricks.net/editor/files/1728975736729870?o=3494755435873649" target="_blank">
    <img src="/Workspace/Repos/APPS/SRC/source/dbx-mitdataprocesses/MITAFO/ATRAN_0050/Jobs/PATRIF_01/08_GEN_MOVIMIENTOS/02_TRANSFERENCIAS/INFONAVIT/Notebooks/assets/NB_PATRIF_MOV_TRANS_TA_01_B.png" style="max-width: 100%; height: auto;"/>
  </a>
</p>

In [0]:
tipo_archivo = conf.p_tipo_archivo
if params.sr_actualiza == '1':
    tipo_archivo = '06'
statement_002 = query.get_statement(
    "PATRIF_MOV_TRANS_TA_01_02.sql",
    CX_PRO_ESQUEMA = conf.CX_PRO_ESQUEMA,
    p_TL_PRO_TRANS_INFONA = conf.p_tl_pro_trans_infona,
    p_SR_FOLIO = params.sr_folio,
    p_SR_SUBPROCESO = params.sr_subproceso,
    p_TIPO_ARCHIVO = 
    tipo_archivo,
    hints="/*+ PARALLEL(4) */",
)

In [0]:
db.write_delta(f"DELTA_TMP_MOV_TRANS_CON_INTERESES_{params.sr_folio}", db.read_data("default", statement_002), "overwrite")
if conf.debug:
    display(db.read_delta(f"DELTA_TMP_MOV_TRANS_CON_INTERESES_{params.sr_folio}"))

<p style="display: flex; gap: 10px;">
  <a href="https://adb-3494755435873649.9.azuredatabricks.net/editor/files/1728975736729878?o=3494755435873649" target="_blank">
    <img src="/Workspace/Repos/APPS/SRC/source/dbx-mitdataprocesses/MITAFO/ATRAN_0050/Jobs/PATRIF_01/08_GEN_MOVIMIENTOS/02_TRANSFERENCIAS/INFONAVIT/Notebooks/assets/NB_PATRIF_MOV_TRANS_TA_01_C.png" style="max-width: 100%; height: auto;"/>
  </a>
</p>

# Reglas para `FTN_MONTO_PESOS`
```
IF FCN_ID_TIPO_SUBCTA = p_tipo_subcta15 THEN
    resultado = FTN_MONTO_PESOS + NullToZero(FTN_INTER_SALDO_VIVI97_ULT)
ELSE
    resultado = FTN_MONTO_PESOS + NullToZero(FTN_INTER_SALDO_VIVI92)
END IF
```

In [0]:
statement_003 = query.get_statement(
    "PATRIF_MOV_TRANS_TA_01_03.sql",
    CATALOG_SCHEMA = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}",
    DELTA_TMP_MOV_TRANS_SALDOS = f"DELTA_TMP_MOV_TRANS_SALDOS_{params.sr_folio}",
    DELTA_TMP_MOV_TRANS_CON_INTERESES = f"DELTA_TMP_MOV_TRANS_CON_INTERESES_{params.sr_folio}",
)

In [0]:
db.write_delta(f"DELTA_500_SALDO_{params.sr_folio}", db.sql_delta(statement_003), "overwrite")
if conf.debug:
    display(db.read_delta(f"DELTA_500_SALDO_{params.sr_folio}"))

In [0]:
CleanUpManager.cleanup_notebook(locals())